In [26]:
from IPython.core.display import display, HTML, Markdown

In [46]:
def is_subseq(s, subs):
    """
    bool: verifica se subs è sottosequenza di s.
    """
    found = 0
    pos_r = 0
    while pos_r < len(s):
        if s[pos_r] == subs[found]:
            found += 1
            if found >= len(subs):
                return True
        pos_r += 1
    return False

def evaluation_format(answ, pt_green,pt_red):
    pt_blue=pt_red-pt_green
    return f"{answ}. Totalizzeresti <span style='color:green'>[{pt_green} safe pt]</span>, \
                                    <span style='color:blue'>[{pt_blue} possible pt]</span>, \
                                    <span style='color:red'>[{pt_red} out of reach pt]</span>.<br>"

"""
Legend of possible sequence types:
      "SC" :  implemented : strettamente crescente
      "ND" :  implemented : non-decrescente
      "SD" :  implemented : strettamente decrescente
      "NC" :  implemented : non-crescente
      "V"  :  implemented : V-sequence (first down then up)
      "A"  :  implemented : A-sequence (first up then down)
      "SV" :  implemented : strict V-sequence (first strict down then strict up)
      "SA" :  implemented : strict A-sequence (first strict up then strict down)
  "ZigZag" :  implemented : parte crescente e poi alterna
  "ZagZig" :  implemented : parte calante e poi alterna
"ZigZagEQ" :  implemented : parte crescente e poi alterna, due consecutivi possono essere uguali
"ZagZigEQ" :  implemented : parte calante e poi alterna, due consecutivi possono essere uguali
"132-free" : not done yet : dal mondo delle permutazioni pattern free per un infinità di problemi in FPT
"""

def is_seq_of_type(s, name_s, seq_type):
    """
    valuta se una sequenza s di nome name_s è di tipo seq_type (vedi tabella subito sopra).
    valore di ritorno (bool, NO_cert_string):
       In caso affermativo il bool ritornato come prima componente è True e la seconda componente è None.
       Altrimenti il bool è False e viene restituita una stringa che riporta una violazione puntuale alla proprietà richiesta.
    """
    if len(s) > 1:
        already_went_down = s[1] < s[0]
        already_went_up = s[1] > s[0]
    for i in range(1,len(s)):
        if s[i] < s[i-1]:
            already_went_down = True
            if seq_type=="V" and already_went_up:
                return (0,f"L'elemento {s[i]} in posizione {i+1} è minore dell'elemento {s[i-1]} in posizione {i} come anche l'elemento {s[i-2]} in posizione {i-1}.")
            if seq_type in {"SC","ND"} or (seq_type in {"ZigZag","ZigZagEQ"} and s[i]%2 == 1) or (seq_type in {"ZagZig","ZagZigEQ"} and s[i]%2 == 0):
                return (0,f"L'elemento {s[i]} in posizione {i+1} è minore dell'elemento {s[i-1]} in posizione {i}.")
        if s[i] > s[i-1]:
            already_went_up = True
            if seq_type=="A" and already_went_down:
                return (0,f"L'elemento {s[i]} in posizione {i+1} è maggiore dell'elemento {s[i-1]} in posizione {i} come anche l'elemento {s[i-2]} in posizione {i-1}.")
            if seq_type in {"SD","NC"} or (seq_type in {"ZagZig","ZagZigEQ"} and s[i]%2 == 1) or (seq_type in {"ZigZag","ZigZagEQ"} and s[i]%2 == 0):
                return (0,f"L'elemento {s[i]} in posizione {i+1} è maggiore dell'elemento {s[i-1]} in posizione {i}.")
        if s[i] == s[i-1] and seq_type in {"SC","SD","SV","SA","ZigZag","ZagZig"}:
            return (0,f"L'elemento {s[i]} in posizione {i+1} non è maggiore dell'elemento {s[i-1]} in posizione {i}.")
    return (1,None)

def is_subseq_of_type(s, name_s, subs, name_subs, subs_type, pt_green, pt_red, forced_ele_pos = None, start_banned_interval = None, end_banned_interval = None):
    """
    Verifica se subs, una sequenza di interi di nome name_subs, è sequenza di tipo subs_type (vedi tabella) e sottosequenza della sequenza s.
    Se forced_ele_pos != None è richiesto che subs contenga l'elemento s[forced_ele_pos].
    Se start_banned_interval,end_banned_interval != None è richiesto che subs eviti il sottointervallo bandito di s.
    Restituisce una stringa contenete la valutazione del certificato subs immesso dallo studente, a tale scopo i parametri pt_green e pt_red mentre pt_blue=pt_red-pt_green è fatto implicito.
    """
    submission_string = f"Hai inserito il certificato ${name_subs}={subs}$."
    submission_string += f"<br>L'istanza era data da ${name_s}={s}$.<br>"

    answ,NO_cert_string = is_seq_of_type(subs, name_subs, subs_type)
    if not answ:
        return submission_string + evaluation_format("No", pt_green,pt_red) + NO_cert_string
    if start_banned_interval != None or end_banned_interval != None:
        assert start_banned_interval != None and end_banned_interval != None
        if forced_ele_pos != None:
            assert forced_ele_pos < start_banned_interval or forced_ele_pos > end_banned_interval
            if forced_ele_pos > end_banned_interval:
               forced_ele_pos -= end_banned_interval 
        aux = s[:start_banned_interval-1] +s[end_banned_interval:]
    if not is_subseq(s, subs):
        return submission_string + evaluation_format("No", pt_green,pt_red) + "La sequenza ${name_subs}$ proposta non è sottosequenza di ${name_s}$."
    if forced_ele_pos != None:
        forced_ele_0basedpos = forced_ele_pos
        found_magic_point = False
        for guess_0basedpos_in_subs in range(len(subs)):
            if subs[guess_0basedpos_in_subs] == s[forced_ele_0basedpos]:
                if is_subseq(s[:forced_ele_0basedpos], subs[:guess_0basedpos_in_subs]) and is_subseq(s[forced_ele_0basedpos:], subs[guess_0basedpos_in_subs:]):
                    found_magic_point = False
        if not found_magic_point:
            return submission_string + evaluation_format("No", pt_green,pt_red) + "La sequenza ${name_subs}$ proposta non è sottosequenza di ${name_s}$."
        
    return submission_string + evaluation_format("Si", pt_green,pt_red)


## Esercizio \[60 pts\]
(poldo) Ricerca di sottosequenze strettamente crescenti di massima lughezza.

Si consideri la seguente sequenza di numeri naturali:

In [32]:
s = [1, 14, 8, 2, 4, 21, 28, 48, 5, 26, 49, 9, 32, 19, 12, 46, 10, 7, 3, 25, 11, 6, 29, 39, 44, 13]

In [33]:
print(s)

[1, 14, 8, 2, 4, 21, 28, 48, 5, 26, 49, 9, 32, 19, 12, 46, 10, 7, 3, 25, 11, 6, 29, 39, 44, 13]



__Richieste__:


1. __[10 pts]__ Trovare una sottosequenza $s1$ strettamente crescente di $s$ che sia la più lunga possibile.

In [38]:
subs1 = [5,49]

In [39]:
display(Markdown(is_subseq_of_type(s, "s", subs1, "subs1", "SC", pt_green=1, pt_red=10, forced_ele_pos=None, start_banned_interval=None, end_banned_interval=None)))

Hai inserito il certificato $subs1=[5, 49]$.<br>L'istanza era data da $s=[1, 14, 8, 2, 4, 21, 28, 48, 5, 26, 49, 9, 32, 19, 12, 46, 10, 7, 3, 25, 11, 6, 29, 39, 44, 13]$.<br>Si. Totalizzeresti <span style='color:green'>[1 safe pt]</span>,                                     <span style='color:blue'>[9 possible pt]</span>,                                     <span style='color:red'>[10 out of reach pt]</span>.<br>

2. __[10 pts]__ Trovare una sottosequenza $s2$ strettamente decrescente di $s$ che sia la più lunga possibile.


In [40]:
subs2 = [5,49]

In [42]:
display(Markdown(is_subseq_of_type(s, "s", subs2, "subs2", "SD", pt_green=1, pt_red=10, forced_ele_pos=None, start_banned_interval=None, end_banned_interval=None)))

Hai inserito il certificato $subs2=[5, 49]$.<br>L'istanza era data da $s=[1, 14, 8, 2, 4, 21, 28, 48, 5, 26, 49, 9, 32, 19, 12, 46, 10, 7, 3, 25, 11, 6, 29, 39, 44, 13]$.<br>No. Totalizzeresti <span style='color:green'>[1 safe pt]</span>,                                     <span style='color:blue'>[9 possible pt]</span>,                                     <span style='color:red'>[10 out of reach pt]</span>.<br>L'elemento 49 in posizione 2 è maggiore dell'elemento 5 in posizione 1.

3. __[10 pts]__ Trovare la più lunga sottosequenza crescente che includa l'elemento di valore 7

In [23]:
print(s[18-1])

7


In [22]:
subs3 = [5,49]

In [47]:
display(Markdown(is_subseq_of_type(s, "s", subs3, "subs3", "SC", pt_green=1, pt_red=10, forced_ele_pos=18, start_banned_interval=None, end_banned_interval=None)))

Hai inserito il certificato $subs3=[5, 49]$.<br>L'istanza era data da $s=[1, 14, 8, 2, 4, 21, 28, 48, 5, 26, 49, 9, 32, 19, 12, 46, 10, 7, 3, 25, 11, 6, 29, 39, 44, 13]$.<br>No. Totalizzeresti <span style='color:green'>[1 safe pt]</span>,                                     <span style='color:blue'>[9 possible pt]</span>,                                     <span style='color:red'>[10 out of reach pt]</span>.<br>La sequenza ${name_subs}$ proposta non è sottosequenza di ${name_s}$.

4. __[10 pts]__ Una sequenza è detta una _V-sequenza_ se cala fino ad un certo punto, e da lì in poi cresce sempre. Trovare la più lunga V-sequenza che sia una sottosequenza della sequenza data

In [ ]:
subs4 = [5,49]

In [49]:
display(Markdown(is_monotone_subseq(s, subs4, "subs_4", "SD")))
display(Markdown(is_subseq_of_type(s, "s", subs4, "subs4", "SD", pt_green=1, pt_red=10, forced_ele_pos=18, start_banned_interval=None, end_banned_interval=None)))

NameError: name 'subs4' is not defined

5. __[20 pts]__ Qual è il minor numero possibile di colori _C_ per colorare gli elementi della sequenza in input in modo che, per ogni colore, la sottosequenza degli elementi di quel colore sia monotona non crescente? Specificare per ogni elemento il colore (come colori, usare i numeri da 1 a _C_)
